<a href="https://colab.research.google.com/github/dareoyeleke/Demand-Index-Volatility-Across-Restaurant-Operations/blob/main/Restaurant_Demand_Index_Volatility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np



In [ ]:
sales_report = pd.read_csv('/content/restaurant_sales_report.csv')
yelp_reviews = pd.read_csv('/content/yelp_restaurant_reviews.csv')
sales_report

In [ ]:
print(sales_report.columns)
print(yelp_reviews.columns)

In [ ]:
sales_report['visit_date'] = pd.to_datetime(sales_report['visit_date'])
yelp_reviews['review_date'] = pd.to_datetime(yelp_reviews['review_date'])
sales_report['visit_month'] = sales_report['visit_date'].dt.to_period('M')
yelp_reviews['review_month'] = yelp_reviews['review_date'].dt.to_period('M')
sales_report['quantity_sold'].dtype

In [ ]:
sales_report['item_revenue'] = (sales_report['order_value'] * sales_report['quantity_sold'])

In [ ]:
monthly_sales = (
    sales_report.groupby(['location_id', 'visit_month'])
    .agg(
        monthly_revenue=('item_revenue', 'sum'),
        avg_unit_price=('order_value', 'mean'),
        avg_items_per_txn=('quantity_sold', 'mean'),
        promo_ratio=('has_promotion', 'mean'),
        restaurant_type=('restaurant_type', lambda x: x.mode()[0] if not x.mode().empty else None),
        weather_condition=('weather_condition', lambda x: x.mode()[0] if not x.mode().empty else None),
        special_event=('special_event', lambda x: x.mode()[0] if not x.mode().empty else False)
    )
    .reset_index()
)


In [ ]:
monthly_sales['revenue_change'] = (monthly_sales
                                   .groupby('location_id')['monthly_revenue']
                                   .pct_change())
monthly_sales['revenue_volatility'] = monthly_sales['revenue_change'].abs()

In [ ]:
monthly_sales['rolling_volatility'] = (
    monthly_sales
    .groupby('location_id')['revenue_volatility']
    .rolling(window=3, min_periods=2)
    .mean()
    .reset_index(level=0, drop=True)
)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
monthly_sales['normalized_volatility'] = scaler.fit_transform(
    monthly_sales[['rolling_volatility']]
)

In [ ]:
monthly_sales['revenue_volatility_index'] = (
    monthly_sales['normalized_volatility']
)

In [ ]:
type_volatility = (monthly_sales
                   .groupby('restaurant_type')
.agg(
    avg_volatility = ('normalized_volatility', 'mean'),
    median_volatility = ('normalized_volatility', 'median'),
    volatility_std = ('normalized_volatility', 'std'),
    location_count = ('location_id', 'nunique'))
    .reset_index()
    .sort_values(by= 'avg_volatility', ascending= False)
)
type_volatility.to_csv('restaurant_type_volatility.csv')

In [ ]:
volatility_across_restaurants = monthly_sales.groupby('restaurant_type')['normalized_volatility'].describe()
volatility_across_restaurants.to_csv('restaurant_volatility.csv')
volatility_across_restaurants

In [ ]:
# The promo_ratio is now calculated in the initial aggregation of monthly_sales.
# This cell's previous content was causing a ValueError due to duplicated rows.
monthly_sales

In [ ]:
def assign_promo_segment(promo_ratio):
    if pd.isna(promo_ratio):
        return 'Unknown'
    elif promo_ratio == 0:
        return 'No Promotion'
    elif promo_ratio <= 0.30:
        return 'Low Promotion'
    elif promo_ratio <= 0.70:
        return 'Medium Promotion'
    else:
        return 'High Promotion'

monthly_sales['promo_segment'] = (
    monthly_sales['promo_ratio']
    .apply(assign_promo_segment)
)


In [ ]:
'''
  Promotion intensity is measured as a ratio of promotional transactions,
  then categorized into promotion segments to distinguish organic demand from promotion-driven demand.
'''
monthly_sales.groupby('promo_segment')['promo_ratio'].describe()

In [ ]:
monthly_sales.to_csv('monthly_sales.csv')

In [ ]:
location_risk = (
    monthly_sales
    .groupby('location_id')
    .agg(
        avg_rvi=('normalized_volatility', 'mean'),
        restaurant_type=('restaurant_type', 'first'),
        dominant_promo_segment=('promo_segment', lambda x: x.mode()[0])
    )
    .reset_index()
)
location_risk

In [ ]:
high_risk_cutoff = location_risk['avg_rvi'].quantile(0.75)
medium_risk_cutoff = location_risk['avg_rvi'].quantile(0.40)
location_risk

In [ ]:
def assign_risk_tier(avg_rvi):
    if avg_rvi >= high_risk_cutoff:
        return 'High Risk'
    elif avg_rvi >= medium_risk_cutoff:
        return 'Medium Risk'
    else:
        return 'Low Risk'


In [ ]:
def risk_explanation(row):
    if row['risk_tier'] == 'High Risk':
        if row['dominant_promo_segment'] == 'High Promotion':
            return 'High volatility likely driven by promotion-dependent demand spikes'
        elif row['dominant_promo_segment'] == 'Medium Promotion':
            return 'Volatility may be masked by frequent promotions; experience consistency review recommended'
        else:
            return 'High volatility without promotion reliance suggests experience or operational instability'

    elif row['risk_tier'] == 'Medium Risk':
        if row['dominant_promo_segment'] == 'High Promotion':
            return 'Demand stabilized by promotions but vulnerable to discount reduction'
        elif row['dominant_promo_segment'] == 'Medium Promotion':
            return 'Moderate volatility likely influenced by contextual factors'
        else:
            return 'Generally stable demand with occasional peak-period sensitivity'

    else:  # Low Risk
        if row['dominant_promo_segment'] == 'High Promotion':
            return 'Promotions used strategically without destabilizing demand'
        elif row['dominant_promo_segment'] == 'Medium Promotion':
            return 'Balanced promotion usage supporting consistent demand'
        else:
            return 'Strong organic demand with high experience consistency'
location_risk

In [ ]:
location_risk['risk_tier'] = location_risk['avg_rvi'].apply(assign_risk_tier)
location_risk['risk_explanation'] = location_risk.apply(
    risk_explanation,
    axis=1
)
location_risk.to_csv('location_risk.csv')

In [ ]:
sales_report['weather_condition'].unique()

In [ ]:
monthly_sales.to_csv('monthly_sales.csv')

In [ ]:
monthly_sales['weather_condition'].unique()

In [ ]:
stress_weather = ['Rainy']

monthly_sales['weather_stress'] = monthly_sales['weather_condition'].apply(
    lambda x: 'Stress' if x in stress_weather else 'Normal'
)
monthly_sales['weather_stress'].value_counts()

monthly_sales['event_stress'] = monthly_sales['special_event'].apply(
    lambda x: 'Event' if x else 'No Event'
)
monthly_sales['event_stress'].value_counts()

In [ ]:
weather_volatility = (
    monthly_sales
    .groupby('weather_stress')
    .agg(
        avg_volatility = ('revenue_volatility_index', 'mean'),
        median_volatility = ('revenue_volatility_index', 'median'),
        count = ('revenue_volatility_index', 'count')
    )
    .reset_index()
)
weather_volatility.to_csv('weather_volatility.csv')

In [ ]:
monthly_sales

In [ ]:
sales_report['weekday'] = sales_report['visit_date'].dt.day_name()

In [ ]:

weekday_order = [
    'Monday', 'Tuesday', 'Wednesday',
    'Thursday', 'Friday', 'Saturday', 'Sunday'
]
sales_report['weekday'] = pd.Categorical(
    sales_report['weekday'],
    categories=weekday_order,
    ordered=True
)
sales_report

In [2]:
# Ran out of Ram so i downloaded and loaded sales report from here
sales_report = pd.read_csv('/content/sales_report.csv')
# monthly_sales = pd.read_csv('/content/monthly_sales.csv')

In [3]:
daily_revenue = (sales_report
                 .groupby(['location_id', 'visit_date', 'weekday', 'restaurant_type'])
                 .agg(daily_revenue=('item_revenue','sum'))
                 .reset_index()

)
daily_revenue

,location_id,visit_date,weekday,restaurant_type,daily_revenue
0,1,2024-01-01,Monday,Food Stall,6117.85
1,1,2024-01-05,Friday,Food Stall,4791.60
2,1,2024-01-12,Friday,Food Stall,6218.17
3,1,2024-01-15,Monday,Food Stall,6076.92
4,1,2024-01-16,Tuesday,Food Stall,5940.42
...,...,...,...,...,...
7638,50,2024-12-22,Sunday,Food Stall,5885.10
7639,50,2024-12-23,Monday,Food Stall,2778.47
7640,50,2024-12-24,Tuesday,Food Stall,617.05
7641,50,2024-12-28,Saturday,Food Stall,3321.15


In [4]:
daily_revenue['pct_change'] = (
    daily_revenue
    .groupby('location_id')['daily_revenue']
    .pct_change()
)

daily_revenue['volatility'] = daily_revenue['pct_change'].abs()

In [ ]:
weekday_volatility = (
    daily_revenue
    .groupby('weekday')
    .agg(
        avg_volatility=('volatility', 'mean'),
        median_volatility=('volatility', 'median'),
        count=('volatility', 'count')
    )
    .reset_index()
)

In [5]:
daily_revenue['safe_revenue'] = daily_revenue['daily_revenue'].replace(0, pd.NA)
daily_revenue

,location_id,visit_date,weekday,restaurant_type,daily_revenue,pct_change,volatility,safe_revenue
0,1,2024-01-01,Monday,Food Stall,6117.85,NaN,NaN,6117.85
1,1,2024-01-05,Friday,Food Stall,4791.60,-0.216784,0.216784,4791.6
2,1,2024-01-12,Friday,Food Stall,6218.17,0.297723,0.297723,6218.17
3,1,2024-01-15,Monday,Food Stall,6076.92,-0.022716,0.022716,6076.92
4,1,2024-01-16,Tuesday,Food Stall,5940.42,-0.022462,0.022462,5940.42
...,...,...,...,...,...,...,...,...
7638,50,2024-12-22,Sunday,Food Stall,5885.10,-0.149594,0.149594,5885.1
7639,50,2024-12-23,Monday,Food Stall,2778.47,-0.527881,0.527881,2778.47
7640,50,2024-12-24,Tuesday,Food Stall,617.05,-0.777917,0.777917,617.05
7641,50,2024-12-28,Saturday,Food Stall,3321.15,4.382303,4.382303,3321.15


In [6]:
daily_revenue['pct_change'] = (
    daily_revenue
    .groupby('location_id')['safe_revenue']
    .pct_change()
)


/tmp/ipython-input-3116708185.py:4: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .pct_change()


In [7]:

daily_revenue['pct_change'] = daily_revenue['pct_change'].replace([np.inf, -np.inf], np.nan)


/tmp/ipython-input-2862143680.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  daily_revenue['pct_change'] = daily_revenue['pct_change'].replace([np.inf, -np.inf], np.nan)


In [8]:
daily_revenue['volatility'] = daily_revenue['pct_change'].abs()
daily_revenue.to_csv('daily_revenue.csv')

In [ ]:
weekday_volatility = (
    daily_revenue
    .groupby('weekday')
    .agg(
        avg_volatility=('volatility', 'mean'),
        median_volatility=('volatility', 'median'),
        count=('volatility', 'count')
    )
    .reset_index()
)
weekday_volatility.to_csv('weekday_volatility.csv')

In [ ]:
weekday_map = {
    'Monday': 'Weekday',
    'Tuesday': 'Weekday',
    'Wednesday': 'Weekday',
    'Thursday': 'Weekday',
    'Friday': 'Weekday',
    'Saturday': 'Weekend',
    'Sunday': 'Weekend'
}

sales_report['day_type'] = sales_report['weekday'].map(weekday_map)


In [10]:
weekday_map = {
    'Monday': 'Weekday',
    'Tuesday': 'Weekday',
    'Wednesday': 'Weekday',
    'Thursday': 'Weekday',
    'Friday': 'Weekday',
    'Saturday': 'Weekend',
    'Sunday': 'Weekend'
}

daily_revenue['day_type'] = daily_revenue['weekday'].map(weekday_map)
daily_revenue.to_csv('daily_revenue.csv')

In [ ]:
daytype_volatility = (
    daily_revenue
    .groupby('day_type')
    .agg(
        avg_volatility=('volatility', 'mean'),
        median_volatility=('volatility', 'median'),
        observation_count=('volatility', 'count')
    )
    .reset_index()
)
daytype_volatility

In [ ]:
print(daily_revenue.columns)
print(sales_report.columns)

In [ ]:
# sales_report = pd.merge(sales_report, daily_revenue, on=['visit_date', 'location_id', 'weekday', ''], how='left')
sales_report

In [ ]:
type_volatility = (
    daily_revenue
    .groupby('restaurant_type')
    .agg(
        avg_volatility=('volatility', 'mean'),
        median_volatility=('volatility', 'median'),
        volatility_std=('volatility', 'std'),
        observation_count=('volatility', 'count')
    )
    .reset_index()
    .sort_values(by='avg_volatility', ascending=False)
)
type_volatility.to_csv('daily_revenue_type_vol.csv')